# Find all the types of documents posible in supercias
### We need to define how many types documents and their frequency in supercias

In [2]:
packages = ['pymongo', 'gridfs', 'pandas', 'pymupdf', 'pytesseract', 'pdf2image', 'openai', 'python-dotenv']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
import pandas as pd
from functions.pdf_operations import PDFOperations
from functions.prompt_openai import query_openai
#endpoint = '10.0.10.5:27017' # use vpn
endpoint = '192.168.1.10:27017' # local lan
#endpoint = '192.168.229.55:27017'
database = 'supercias'
collection = 'companies'
# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
# get collection
collection = db[collection]
# where the pdfs will be stored 
pdfs_path = '../storage/pdfs/'
# pdf operator
pdf_op = PDFOperations(db, 'companies', pdfs_path)
#find pdf file
companies_cursor = collection.find()


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


Let seperate the docuemt by the type they can belong to: Generales, Economicos  and Judiciales

In [3]:
# make a new collection to store the documents cleaned docs
clean_collection = db['clean_companies']
missing_collection = db['missing_companies']

# now lets get all the documents for every company
companies = collection.find()
def clean_company(company, clean_collection, missing_collection):
    # get the _id
    id = company['_id']
    del company['_id']
    # check if the ruc in already in the clean collection
    if clean_collection.find_one({'ruc': company['ruc']}): return
    # create a new list to store the cleaned documents and the missing ones
    clean_Documentos_generales = []
    clean_Documentos_economicos = []
    clean_Documentos_juridicos = []
    missing_Documentos_generales = []
    missing_Documentos_economicos = []
    missing_Documentos_juridicos = []
    # check if we have documentes online, otherwise add the company to the missing collection
    if 'Documentos online' not in company:
        missing_collection.insert_one(company)
        return
    # get the documents
    documentos = company['Documentos online']['downloaded']
    # for every document check if it exists on the gridfs
    for documento in documentos['DocumentosGenerales']:
        if pdf_op.exists(documento):
            clean_Documentos_generales.append(documento)
        else:
            missing_Documentos_generales.append(documento)
    for documento in documentos['DocumentosEconomicos']:
        if pdf_op.exists(documento):
            clean_Documentos_economicos.append(documento)
        else:
            missing_Documentos_economicos.append(documento)
    for documento in documentos['DocumentosJuridicos']:
        if pdf_op.exists(documento):
            clean_Documentos_juridicos.append(documento)
        else:   
            missing_Documentos_juridicos.append(documento)
    # append the cleaned documents to the company and push to the clean collection
    del company['Documentos online']
    company['Documentos'] = {}
    company['Documentos']['Generales'] = clean_Documentos_generales
    company['Documentos']['Economicos'] = clean_Documentos_economicos
    company['Documentos']['Juridicos'] = clean_Documentos_juridicos
    clean_collection.insert_one(company)
    # if we have missing documents append the company to the missing collection
    company['Documentos']['Generales'] = missing_Documentos_generales
    company['Documentos']['Economicos'] = missing_Documentos_economicos
    company['Documentos']['Juridicos'] = missing_Documentos_juridicos
    missing_collection.insert_one(company)

    

let's run this function single threaded to see if it works

In [ ]:
for company in companies:
    clean_company(company, clean_collection, missing_collection)

let's run this functino multitreaded to be able to finish today

In [4]:
import concurrent.futures
import traceback

# Multithreading implementation
def clean_companies_in_parallel(companies, clean_collection, missing_collection, max_workers=5):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks for each row to the executor, passing the collection explicitly
        futures = {executor.submit(clean_company, company, clean_collection, missing_collection) for company in companies}
        # Optionally, you can handle results or exceptions here
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # We can use this to raise exceptions if any occurred in the threads
            except Exception as e:
                # remove row from df if error
                print(f"Exception in thread: {e}")
                traceback.print_exc()


# Adjust the number of threads as needed
max_threads = 10  # Example: Use 5 threads
# Execute the function with multithreading, passing the collection explicitly
clean_companies_in_parallel(companies, clean_collection, missing_collection, max_threads)